In [1]:
from pyomo.environ import *
from brooks import *
import pandas as pd

In [2]:
team_data = pd.read_excel("project_data_model.xlsx", 
                          sheet_name="Sheet2")


Define data

In [3]:
W = list(team_data["Person ID"])
S = ["Analyst","Application Architect","Architect","Communication Specialists","Dev/QA Manager (DevOps)","Developer/Engineer","Governance","Operations Specialist","Product Owner","Release Management","Scrum Master","Solution Architect","Tester"]
E = ["advanced","associate","intermediate","senior"]
P = ["Build","Plan","Run","Scrum"]
Z = ["ET", "CT", "PT"]
Y = ["0-5","10-20","20+","5-10"]
A = ["basic","highly experienced and certified","training/experience"]
T = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
#timezone = dict(zip(W,list(team_data["Time Zone"])))
teams_max = 14
tsplus = 2
tsminus = 2
feplus = 1
feminus = 1
fpplus = 1
fpminus = 1
function_traget = 3
agile_weekly_cost = 2066.67
time_zone_cost = 4
team_size_cost = 14
function_exp_cost = 6
product_owner_cost = 3
area_cost = 5

Declare a model object

In [4]:
model = Model()

Declare non negative for variables

In [5]:
if hasattr(model,'x'):
    model.delete(model.x)
model.x = Var(W,T, domain = Binary)

In [6]:
if hasattr(model,'y'):
    model.delete(model.y)
model.y = Var(T,Z, domain = Binary)

In [7]:
if hasattr(model,'z'):
    model.delete(model.z)
model.z = Var(T, domain = Binary )

In [8]:
if hasattr(model,'e'):
    model.delete(model.e)
model.e = Var(T,E, domain = NonNegativeIntegers)

In [9]:
if hasattr(model,'f'):
    model.delete(model.f)
model.f = Var(T,P, domain = NonNegativeIntegers)

In [10]:
if hasattr(model,'g'):
    model.delete(model.g)
model.g = Var(Z,T, domain = Binary )

In [11]:
if hasattr(model,'he'):
    model.delete(model.he)
model.he = Var(T, domain = Binary )

In [12]:
if hasattr(model,'te'):
    model.delete(model.te)
model.te = Var(T, domain = NonNegativeIntegers )

In [13]:
if hasattr(model,'tsplus'):
    model.delete(model.tsplus)
model.tsplus = Var(T, domain = NonNegativeReals, bounds = (0,2))

In [14]:
if hasattr(model,'tsminus'):
    model.delete(model.tsminus)
model.tsminus = Var(T, domain = NonNegativeReals, bounds = (0,2))

In [15]:
if hasattr(model,'feplus'):
    model.delete(model.feplus)
model.feplus = Var(T,E,domain = NonNegativeIntegers, bounds = (0,1))

In [16]:
if hasattr(model,'feminus'):
    model.delete(model.feminus)
model.feminus = Var(T,E,domain = NonNegativeIntegers, bounds = (0,1))

In [17]:
if hasattr(model,'fpplus'):
    model.delete(model.fpplus)
model.fpplus = Var(T,P,domain = NonNegativeIntegers, bounds = (0,1))

In [18]:
if hasattr(model,'fpminus'):
    model.delete(model.fpminus)
model.fpminus = Var(T,P,domain = NonNegativeIntegers, bounds = (0,1))

Define the objective function

In [19]:
if hasattr(model, 'cost_objective'):
    model.delete(model.cost_objective)

model.cost_objective = Objective(expr = (agile_weekly_cost*sum(model.z[j]  for j in T))+ (time_zone_cost*sum(model.y[j,k] for j in T for k in Z))+ (team_size_cost*sum(model.tsplus[j] + model.tsminus[j] for j in T)) + (function_exp_cost*sum(model.feplus[j,k] + model.feminus[j,k] for j in T for k in E))+ (product_owner_cost*sum(model.x[i,j] for i in W for j in T )) + (area_cost* sum(model.fpplus[j,k]+model.fpminus[j,k] for j in T for k in P if k!="Scrum"))
 ,sense=minimize)

Specifying Allocation constraint

In [20]:
# All workers must be allocated constraint
if hasattr(model, 'allocation_constraint'):
    model.delete(model.allocation_constraint)
model.allocation_constraint = ConstraintList()
for i in W:
    if S != "Scrum Master":
        model.allocation_constraint.add(sum(model.x[i,j] for j in T) == 1)


Team size constraint

In [21]:
# 1st Constraint
if hasattr(model, 'teamsize_constraint'):
    model.delete(model.teamsize_constraint)
model.teamsize_constraint = ConstraintList()
for i in W:
    model.teamsize_constraint.add(sum(model.x[i,j] for j in T) <= (10+tsplus-tsminus))

Functional expertise diversity constraint

In [22]:
# 2nd DE Constraint
if hasattr(model, 'Exp_Div_constraint'):
    model.delete(model.Exp_Div_constraint)
model.Exp_Div_constraint = ConstraintList()
for k in E:
    model.Exp_Div_constraint.add(sum(model.e[j,k] for j in T) <= (2.5+feplus-feminus))

Product owner constraint

In [23]:
# 3rd Product Owner Constraint
if hasattr(model, 'Po_constraint'):
    model.delete(model.Po_constraint)
if P == "Y":
    model.Po_constraint = ConstraintList()
    for j in T:
        model.Po_constraint.add(sum(model.x[i,j] for i in W) >= 1)

Functional constraint

In [24]:
# 4th funtional Constraint
if hasattr(model, 'functional_constraint'):
    model.delete(model.functional_constraint)
model.functional_constraint = ConstraintList()
for j in T:
    for k in P:
        if k != "Scrum":
            if P == k:
                model.functional_constraint.add(sum(model.x[i,j] for i in W  ) == function_traget+model.fpplus[j,k]-model.fpminus[j,k])

In [25]:
# 5th Time zone Constraint
#if hasattr(model, 'Time_constraint'):
#    model.delete(model.Time_constraint)
#model.Time_constraint = ConstraintList()
#for j in T:
#    for i in W:
#        model.Time_constraint.add((model.x[i,j]) <= model.y[j,timezone[i]])

Agile constraint

In [26]:
# 6th Agile constraint
if hasattr(model, 'he_flip_off'):
    model.delete(model.he_flip_off)
model.he_flip_off = ConstraintList()
for j in T:
    for i in W:
        if A == "highly experienced and certified":
            model.he_flip_off.add(sum(model.x[i,j])>= model.he[j])

Agile constraint for highly experienced 

In [27]:
if hasattr(model, 'te_flip_off'):
    model.delete(model.te_flip_off)
for j in T:
    model.te_flip_off= Constraint(expr = sum(model.x[i,j] for i in W if A in
                              ["highly experienced and certified","training/experience"])- 0.3* sum(model.x[i,j] for i in W) >= -12*(1-model.te[j]))
    


    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractSimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractSimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractSimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (ty

Scrum Master constraint

In [28]:
# 8th scrum master Constraint
if hasattr(model, 'scrum_master_constraint'):
    model.delete(model.scrum_master_constraint)
model.scrum_master_constraint = ConstraintList()
for j in T:
    if S == "Scrum Master":
        model.scrum_master_constraint.add(sum(model.x[i,j] for i in W) <= 2 and (model.x[i,j]for i in W) >=1)

Specify solver and solve

In [29]:
solver = SolverFactory('gurobi')
status = solver.solve(model)

Get status

In [30]:
print('status = %s' % status)

status = 
Problem: 
- Name: x2493
  Lower bound: 429.0
  Upper bound: 429.0
  Number of objectives: 1
  Number of constraints: 292
  Number of variables: 2340
  Number of binary variables: 2058
  Number of integer variables: 2311
  Number of continuous variables: 29
  Number of nonzeros: 4205
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.015626907348632812
  Error rc: 0
  Time: 0.18144750595092773
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



## Result

Get solution and objective function value

In [31]:
for i in W:
    for j in T:
        if value(model.x[i,j])>0:
            print('%s = %f' % (model.x[i,j], value(model.x[i,j])))
print("objective = %f" % value(model.cost_objective))

x[1,5] = 1.000000
x[2,8] = 1.000000
x[3,1] = 1.000000
x[4,5] = 1.000000
x[5,2] = 1.000000
x[6,7] = 1.000000
x[7,7] = 1.000000
x[8,11] = 1.000000
x[9,12] = 1.000000
x[10,4] = 1.000000
x[11,4] = 1.000000
x[12,1] = 1.000000
x[13,3] = 1.000000
x[14,2] = 1.000000
x[15,6] = 1.000000
x[16,12] = 1.000000
x[17,2] = 1.000000
x[18,8] = 1.000000
x[19,10] = 1.000000
x[20,4] = 1.000000
x[21,10] = 1.000000
x[22,6] = 1.000000
x[23,2] = 1.000000
x[24,11] = 1.000000
x[25,2] = 1.000000
x[26,1] = 1.000000
x[27,5] = 1.000000
x[28,8] = 1.000000
x[29,3] = 1.000000
x[30,2] = 1.000000
x[31,8] = 1.000000
x[32,2] = 1.000000
x[33,9] = 1.000000
x[34,2] = 1.000000
x[35,8] = 1.000000
x[36,11] = 1.000000
x[37,13] = 1.000000
x[38,8] = 1.000000
x[39,12] = 1.000000
x[40,6] = 1.000000
x[41,10] = 1.000000
x[42,8] = 1.000000
x[43,2] = 1.000000
x[44,11] = 1.000000
x[45,2] = 1.000000
x[46,3] = 1.000000
x[47,13] = 1.000000
x[48,3] = 1.000000
x[49,5] = 1.000000
x[50,11] = 1.000000
x[51,1] = 1.000000
x[52,4] = 1.000000
x[53,6] 